In [ ]:
# Create Stash app client

import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
# Utility functions
def create_modified_filter(data_quality_filter):
    depth = data_quality_filter['object_filter']['tags']['value']['depth']
    included_tag_ids = [tag['id'] for tag in data_quality_filter['object_filter']['tags']['value']['items']]
    excluded_tag_ids = [tag['id'] for tag in data_quality_filter['object_filter']['tags']['value']['excluded']]

    # Create the modified filter
    modified_filter = {
        'tags': {
            'modifier': 'INCLUDES_ALL',
            'value': included_tag_ids,
            'depth': depth,
            'excludes': excluded_tag_ids,
        }
    }
    return modified_filter

def get_scene_count(filter):
    result = stash.call_GQL("""query FindScenes($scene_filter: SceneFilterType) {
        findScenes(scene_filter: $scene_filter) {
            count
        }
    }""", variables={"scene_filter": filter})
    return result['findScenes']['count']

In [ ]:
# Get all saved filters
saved_filters = stash.call_GQL("""query FindSavedFilters {
    findSavedFilters {
        id
        mode
        name
        filter
        object_filter
        ui_options
    }
}""")

In [ ]:
# Get all data quality filters
data_quality_filters = [filter for filter in saved_filters['findSavedFilters'] if 'data quality' in filter['name'].lower()]

In [ ]:
# Loop through all data quality filters and print out name and count of matching scenes
sorted_data_quality_filters = sorted(data_quality_filters, key=lambda x: x['name'])
for filter in sorted_data_quality_filters:
    name = filter['name']
    modified_filter = create_modified_filter(filter)
    scene_count = get_scene_count(modified_filter)
    print(f"{name}: {scene_count} scenes")


# Additional checks for performer genders

In [ ]:
included_tag_names = ["Twosome", "Twosome (Straight)"]
included_tag_ids = [stash.find_tag(name)["id"] for name in included_tag_names]
excluded_tag_names = ["Full Movie"]
excluded_tag_ids = [stash.find_tag(name)["id"] for name in excluded_tag_names]

twosome_bg_scenes = stash.find_scenes(
    {
        "tags": {
            "modifier": "INCLUDES_ALL",
            "value": included_tag_ids,
            "excludes": excluded_tag_ids
        }
    },
    # filter={"per_page": 100},
    fragment="id title performers { id name gender }"
)

def check_performer_genders(scene):
    has_male = any(performer['gender'] == 'MALE' for performer in scene['performers'])
    has_female = any(performer['gender'] == 'FEMALE' for performer in scene['performers'])
    return not (has_male and has_female)

scenes_with_incorrect_genders = [
    scene for scene in twosome_bg_scenes
    if len(scene['performers']) == 2 and check_performer_genders(scene)
]

print(f"Scenes with tags {included_tag_names} with two performers but without one MALE and one FEMALE performer:")
for scene in scenes_with_incorrect_genders:
    print(f"ID: {scene['id']}, Title: {scene['title']}")
    print("Performers:")
    for performer in scene['performers']:
        print(f"  - {performer['name']} ({performer['gender']})")
    print()


In [ ]:
straight_bg_tag = stash.find_tag("Twosome (Straight)")
straight_bg_tag_id = straight_bg_tag["id"]
lesbian_gg_tag = stash.find_tag("Twosome (Lesbian)")
lesbian_gg_tag_id = lesbian_gg_tag["id"]

for scene in scenes_with_incorrect_genders:
    refreshed_scene = stash.find_scene(scene["id"])
    updated_tags = refreshed_scene["tags"]
    updated_tag_ids = [tag["id"] for tag in updated_tags]
    updated_tag_ids = [tag_id for tag_id in updated_tag_ids if tag_id is not None and tag_id != straight_bg_tag_id]
    if lesbian_gg_tag_id not in updated_tag_ids:
        updated_tag_ids.append(lesbian_gg_tag_id)
    stash.update_scene({
        "id": scene["id"],
        "tag_ids": updated_tag_ids
    })
    print(f"Updated scene {scene['id']}: Removed 'Twosome (Straight)' tag and added 'Twosome (Lesbian)' tag")
